In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import load_model
import tensorflow as tf
from PIL import Image
import os

In [67]:
class DSS:
    
    def __init__(self, model_path = 'best_head.h5'):
        self.model = load_model(model_path)
        self.pred_dict = {0:'COVID-19', 1:'normal', 2:'pneumonia'}
        try:
            self.hist_data = pd.read_csv('DB')
        except:
            self.hist_data = pd.DataFrame(columns =['Patient_name',
                                                   'Patient_id',
                                                   'Date_of_photo',
                                                   'Shot',
                                                   'Predicted_class',
                                                   'class_probability',
                                                   'actual_results'])
    def __get_image(self):
        name = input('Please insert name of the image: ')
        path = 'images/' + name +'.jpg'
        np_image = Image.open(path)
        np_image = np_image.resize((299, 299), Image.ANTIALIAS)
        np_image = np.array(np_image).astype('float32')/255
        np_image = np.expand_dims(np_image, axis=0)
        return np_image
    
    def __predict_class(self, image):
        prediction = self.model.predict(tf.convert_to_tensor(image))
        predicted_class = np.argmax(prediction)
        class_probability = prediction[0, predicted_class]
        predicted_class = self.pred_dict[predicted_class]
        return predicted_class, class_probability
    
    def __new_patient(self):
        patient_name = input('Please input patient name: ')
        patient_id = input('Please input patient id: ')
        date = input('Please input date of X-Ray shot: ')
        img = self.__get_image()
        predicted_class, class_probability = self.__predict_class(img)
        df = pd.DataFrame({'Patient_name':[patient_name],
                           'Patient_id':[patient_id],
                           'Date_of_photo':[date],
                           'Shot':[img],
                           'Predicted_class':[predicted_class],
                           'class_probability':[class_probability]})
        self.hist_data = self.hist_data.append(df, ignore_index = True)
        print('Prediction is: ' + str(predicted_class)
              +'\nProbability of the prediction: ' + str(class_probability))
        
    def __edit_patient_info(self):
        row_id = input('Please chose row where you want to update the actual results: ')
        act_res = input('Please insert actual results: ')
        self.hist_data.iloc[int(row_id), -1] = act_res
        print('Updated!')
        
    def __finish(self):
        self.hist_data.to_csv('DB', index = False)
        print('DataBase is saved!')
    
    def actions(self):
        stop = False
        while(not stop):
            act = input('Please, input what you want to do: \n     1 -- predict for new patient, \
            \n     2 -- insert actual results for previous patients,\n     3 -- finish work\n')
            if act == '1':
                self.__new_patient()
            elif act == '2':
                self.__edit_patient_info()
            elif act == '3':
                self.__finish()
                stop = True
            else:
                print('Sorry, wrong input!')
         

In [68]:
dss = DSS()
dss.actions()

Please, input what you want to do: 
     1 -- predict for new patient,             
     2 -- insert actual results for previous patients,
     3 -- finish work
1
Please input patient name: Ivanov Ivan
Please input patient id: II
Please input date of X-Ray shot: 2021-06-05
Please insert name of the image: 1
Prediction is: normal
Probability of the prediction: 0.6918721
Please, input what you want to do: 
     1 -- predict for new patient,             
     2 -- insert actual results for previous patients,
     3 -- finish work
1
Please input patient name: Petrov Petr
Please input patient id: PP
Please input date of X-Ray shot: 2021-04-16
Please insert name of the image: 3
Prediction is: pneumonia
Probability of the prediction: 0.63097614
Please, input what you want to do: 
     1 -- predict for new patient,             
     2 -- insert actual results for previous patients,
     3 -- finish work
2
Please chose row where you want to update the actual results: 0
Please insert actual resu

In [69]:
df = pd.read_csv('DB')
df

,Patient_name,Patient_id,Date_of_photo,Shot,Predicted_class,class_probability,actual_results
0,Ivanov Ivan,II,2021-06-05,[[[[0.00784314 0.00784314 0.00784314]\n [0.0...,normal,0.691872,COVID-19
1,Petrov Petr,PP,2021-04-16,[[[[0. 0. 0.]\n [0. 0. 0.]\n [0. 0. 0.]\n ...,pneumonia,0.630976,pneumonia
